<h1><b>Herramientas</b></h1>
Tool calling

Ejercicios

In [ ]:
#Libreria para Gemini
%pip install google-generativeai
#!pip show google-generativeai

In [ ]:
from google.colab import userdata
import google.generativeai as genai
import os
import json
import pandas as pd

In [ ]:
#Accedemos a gemini
genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))

#Revisamos los modelos existentes
for model in genai.list_models():
    print(model.name)

### Funcion para revisar inventario de sucursales

In [ ]:
df_promociones = pd.read_csv('https://storage.googleapis.com/tallerdp_publico/taller02/promociones.csv')
df_inventarios = pd.read_csv('https://storage.googleapis.com/tallerdp_publico/taller02/inventario.csv')
df_politicas = pd.read_csv('https://storage.googleapis.com/tallerdp_publico/taller02/politica_devoluciones.csv')

def revisar_inventario_sucursales(producto: str = None) -> str:
    """
    Devuelve stock y precio de los productos cuyo nombre contenga la
    cadena 'producto'.  Trabaja sobre 'inventario.csv' (una sola tienda).

    Args:
        producto (str): parte del nombre a buscar.  Ej.: 'tenis onboard'

    Returns:
        str (JSON):
            - Si no hay coincidencias: {"mensaje": "..."}
            - Si 1 coincidencia: {"producto": "...", "stock": 44, "precio": 650.62}
            - Si >1: {"producto_consultado": "...", "total_productos": N, "detalles":[...]}
    """
    try:
        df = df_inventarios

        df['product_name'] = df['product_name'].str.lower()

        if producto:
            producto = producto.lower()
            df = df[df['product_name'].str.contains(producto, na=False)]

        if df.empty:
            return json.dumps({"mensaje": "No se encontró inventario para ese producto."})

        df_out = df[["product_name", "stock", "price"]]
        if len(df_out) == 1:
            r = df_out.iloc[0]
            return json.dumps(
                {
                    "producto": r["product_name"],
                    "stock": int(r["stock"]),
                    "precio": float(r["price"]),
                },
                indent=2,
            )

        return json.dumps(
            {
                "producto_consultado": producto,
                "total_productos": len(df_out),
                "detalles": df_out.to_dict(orient="records"),
            },
            indent=2,
        )

    except Exception as e:
        return json.dumps({"error": f"Error en inventario: {str(e)}"})

### Funcion para revisar promociones

In [ ]:
def revisar_promociones(categoria: str = None) -> str:
    try:
        df = df_promociones
        if categoria:
            df = df[df['categoria'].str.contains(categoria, case=False, na=False)]
        if df.empty:
            return json.dumps({"mensaje": "No hay promociones activas para la categoría especificada."})
        return df.to_json(orient="records", indent=2)
    except Exception as e:
        return json.dumps({"error": f"Error al procesar promociones: {str(e)}"})

### Funcion para revisar politica de devoluciones

In [ ]:
def revisar_politica_devoluciones(termino_clave: str = None) -> str:
    try:
        df =  df_politicas
        if not termino_clave or not termino_clave.strip():
            termino_clave = "devolución"

        resultados = df[df.apply(lambda row: termino_clave.lower() in str(row).lower(), axis=1)]

        if resultados.empty:
            return json.dumps({"mensaje": f"No se encontró información sobre '{termino_clave}'."})

        return resultados.to_json(orient='records', indent=2)

    except Exception as e:
        return json.dumps({"error": f"Error al procesar política de devoluciones: {str(e)}"})

### Asociar nombres a funciones

In [ ]:

available_tools = {
    "revisar_inventario_sucursales": revisar_inventario_sucursales,
    "revisar_promociones": revisar_promociones,
    "revisar_politica_devoluciones": revisar_politica_devoluciones
}

### Crear modelo con herramientas habilitadas


In [ ]:
model = genai.GenerativeModel("models/gemini-2.5-flash-preview-05-20",
    tools=list(available_tools.values())
)

In [ ]:
def preguntar_al_agente(pregunta_usuario: str):
    print(f"\n Usuario: {pregunta_usuario}")
    chat = model.start_chat(enable_automatic_function_calling=True)

    try:
        response = chat.send_message(pregunta_usuario)
        print("\n Agente:")
        print(response.text)
    except Exception as e:
        print(f" Error al procesar la pregunta: {e}")

### EJEMPLOS DE USO

In [ ]:
preguntar_al_agente("¿Tienes stock de Sandalias Onboard Dore Mujer?")

In [ ]:
preguntar_al_agente("Tienes Tenis Onboard?")

In [ ]:
preguntar_al_agente("¿Me podrias decir la existencia de Sandalias adidas Adilette Lumia Mujer?")

In [ ]:
preguntar_al_agente("¿Qué promociones hay para lentes?")

In [ ]:
preguntar_al_agente("¿Cuál es la política de devolución si el producto está usado?")

In [ ]:
preguntar_al_agente("¿puedo hacer cambio de producto?")